In [ ]:
%cd /content
# remove first if exists
!rm -rf arabic-poetry-speech-classification
!git clone https://github.com/MagedSaeed/arabic-poetry-speech-classification.git

/content
Cloning into 'arabic-poetry-speech-classification'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 43 (delta 19), reused 31 (delta 11), pack-reused 0
Unpacking objects: 100% (43/43), done.


In [ ]:
# mount the drive to get the datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# put the task path you want to work on here
%cd arabic-poetry-speech-classification/signal-classification

/content/arabic-poetry-speech-classification/signal-classification


In [ ]:
!pip install torchaudio
!pip install transformers
!pip install datasets
!pip install lang_trans
!pip install arabic_reshaper
!pip install python-bidi
!pip install pydub
!pip install soundfile
!pip install jiwer==2.2.0
!pip install PyArabic

     |████████████████████████████████| 3.1 MB 5.0 MB/s 
     |████████████████████████████████| 3.3 MB 51.9 MB/s 
     |████████████████████████████████| 596 kB 51.2 MB/s 
     |████████████████████████████████| 61 kB 433 kB/s 
     |████████████████████████████████| 895 kB 46.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 298 kB 4.9 MB/s 
     |████████████████████████████████| 243 kB 51.1 MB/s 
     |████████████████████████████████| 132 kB 65.8 MB/s 
     |████████████████████████████████| 1.1 MB 46.4 MB/s 
     |████████████████████████████████| 160 kB 59.3 MB/s 
     |████████████████████████████████| 192 kB 55.5 MB/s 
     |████████████████████████████████| 271 kB 48.5 MB/s 
  Created wheel for lang-trans: filename=lang_trans-0.6.0-py3-none-any.whl size=6345 sha256=9aa91dbd73b1b7c438a4e1c9a64fe10c180fce1ee20b28d260184ee16771e63a
  

In [ ]:
import os
import re
import sys
import json
import torch
import jiwer
import logging
import librosa
import datasets
import itertools
import torchaudio
import numpy as np
import transformers
import pandas as pd
from torch import nn
import seaborn as sns
from tqdm import tqdm
import torch.nn as nn
import soundfile as sf
import arabic_reshaper
from pyarabic import araby
from packaging import version
from pydub import AudioSegment
from dataclasses import asdict
from trainer import CTCTrainer
import matplotlib.pyplot as plt
from pydub.utils import mediainfo
from argparse import ArgumentParser
from collections import defaultdict
from torch.nn import functional as F
from contextlib import contextmanager
from bidi.algorithm import get_display
from lang_trans.arabic import buckwalter
from dataclasses import dataclass, field
from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score
from models import Wav2Vec2ClassificationModel
from processors import CustomWav2Vec2Processor
from typing import Any, Dict, List, Optional, Union
from transformers import HfArgumentParser,TrainingArguments
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
from transformers.trainer_utils import get_last_checkpoint, is_main_process
from transformers import is_apex_available,set_seed ,Trainer,Wav2Vec2FeatureExtractor
from arg_parsers import DataTrainingArguments, ModelArguments, DataCollatorCTCWithPadding
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Model,Wav2Vec2PreTrainedModel
% matplotlib inline

In [ ]:
!cp -r /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/All_poems.zip .

In [ ]:
if os.path.exists('dataset'):
  if len(os.listdir('dataset')) == 0:
    os.system('unzip All_poems.zip -d dataset')
else:
  os.system('unzip All_poems.zip -d dataset')

In [ ]:
!mkdir -p dataset_wav

In [ ]:
!cp /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/testset.csv .
!cp /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/trainset.csv .
!cp /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/valset.csv .

In [ ]:
metadata_test_path = 'testset.csv'
metadata_train_path = 'trainset.csv'
metadata_val_path = 'valset.csv'
dataset_folder = 'dataset'
dataset_wav_folder = 'dataset_wav'

In [ ]:
train_metadata = pd.read_csv(metadata_train_path)
test_metadata = pd.read_csv(metadata_test_path)
val_metadata = pd.read_csv(metadata_val_path)

In [ ]:
sample_rates = set()
for file_path in tqdm(list(itertools.chain(
    train_metadata['Utterance name'],
    test_metadata['Utterance name'],
    val_metadata['Utterance name']
  ))):
  complete_path = f'{dataset_folder}/{file_path}'
  complete_wav_path = f'{dataset_wav_folder}/{file_path}'
  # os.system(f'ffmpeg -i {complete_path} {complete_wav_path}')
  audio = AudioSegment.from_file(complete_path)
  sample_rates.add(audio.frame_rate)
  audio.export(f'{dataset_wav_folder}/{file_path}', format='wav')
sample_rates

100%|██████████| 3668/3668 [10:05<00:00,  6.05it/s]


{44100, 48000}

In [ ]:
os.environ["WANDB_DISABLED"] = "true"
if is_apex_available():
    from apex import amp

In [ ]:
if version.parse(torch.__version__) >= version.parse("1.6"):
    _is_native_amp_available = True
    from torch.cuda.amp import autocast

logger = logging.getLogger(__name__)

## getting training arguments

In [ ]:
# testing path poetry-classification
# model_output_path = '--output_dir=poetry-classification'
args_string="""
--model_name_or_path="bakrianoo/sinai-voice-ar-stt"
--output_dir=/content/drive/MyDrive/KFUPM-Master/ICS606/Models/ClassificationModelSinaiFineTuned
--freeze_feature_extractor 
--num_train_epochs="5" 
--per_device_train_batch_size="2" 
--preprocessing_num_workers="1" 
--learning_rate="3e-5" 
--warmup_steps="5" 
--evaluation_strategy="steps"
--save_steps="500" 
--eval_steps="100" 
--save_total_limit="1" 
--logging_steps="50" 
--do_eval=1
--do_train=1
"""
def match_numeric_type(s):
  try:
    return json.loads(s)
  except:
    return s

# construct the args dict from the args string
args_dict = dict()
for line in args_string.splitlines():
  line = line.replace('--','').replace(r'\\','').replace('"','').replace("'",'').strip()
  if line:
    if '=' in line:
      arg_name,arg_value = line.split('=')
      args_dict[arg_name] = arg_value
    else:
      args_dict[line] = ''

# match args to dataclasses
args_class_attrs = defaultdict(list)
for arg_name,arg_value in args_dict.items():
  for ArgClass in (ModelArguments, DataTrainingArguments, TrainingArguments):
    if arg_name in ArgClass.__annotations__:
      args_class_attrs[ArgClass].append(arg_name)

# initialzie args objects
model_args = ModelArguments(**{arg_name:match_numeric_type(args_dict[arg_name]) for arg_name in args_class_attrs[ModelArguments]})
data_args = DataTrainingArguments(**{arg_name:match_numeric_type(args_dict[arg_name]) for arg_name in args_class_attrs[DataTrainingArguments]})
training_args = TrainingArguments(**{arg_name:match_numeric_type(args_dict[arg_name]) for arg_name in args_class_attrs[TrainingArguments]})

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


## logging setup

In [ ]:
# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)
# logger.setLevel(
#     logging.INFO if is_main_process(training_args.local_rank) else logging.WARN
# )
logger.setLevel(
    logging.WARN
)
# Log on each process the small summary:
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f"distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
# Set the verbosity to info of the Transformers logger (on main process only):
if is_main_process(training_args.local_rank):
    transformers.utils.logging.set_verbosity_info()

# Set seed before initializing model.
set_seed(training_args.seed)


12/03/2021 17:42:27 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False


## dataset processing

In [ ]:
train_metadata = pd.read_csv(metadata_train_path)
val_metadata = pd.read_csv(metadata_val_path)
test_metadata = pd.read_csv(metadata_test_path)

train_dataset = Dataset.from_pandas(train_metadata)
eval_dataset = Dataset.from_pandas(val_metadata)
test_dataset = Dataset.from_pandas(test_metadata)

if data_args.max_train_samples is not None:
    train_dataset = train_dataset.select(range(data_args.max_train_samples))

if data_args.max_val_samples is not None:
    eval_dataset = eval_dataset.select(range(data_args.max_val_samples))
# Preprocessing the datasets.
# We need to read the aduio files as arrays and tokenize the targets.
resamplers = {  # The dataset contains all the uncommented sample rates
    48000: torchaudio.transforms.Resample(48000, 16000),
    44100: torchaudio.transforms.Resample(44100, 16000),
    # 32000: torchaudio.transforms.Resample(32000, 16000),
}

labels = {
    bahr: bahr_index
    for bahr_index, bahr in enumerate(sorted(set(train_metadata["Bahr"])))
}
print("labels are:", labels)
print("len:", len(labels))

def speech_file_to_array_fn(batch):
    start = 0
    stop = 20
    srate = 16_000
    speech_array, sampling_rate = torchaudio.load(
        f'dataset_wav/{batch["Utterance name"]}'
    )
    speech_array = speech_array[0]
    batch["speech"] = resamplers[sampling_rate](speech_array).squeeze().numpy()
    batch["sampling_rate"] = srate
    batch["parent"] = labels[batch["Bahr"]]
    return batch

train_dataset = train_dataset.map(
    speech_file_to_array_fn,
    remove_columns=train_dataset.column_names,
    num_proc=data_args.preprocessing_num_workers,
)
eval_dataset = eval_dataset.map(
    speech_file_to_array_fn,
    remove_columns=eval_dataset.column_names,
    num_proc=data_args.preprocessing_num_workers,
)

test_dataset = test_dataset.map(
    speech_file_to_array_fn,
    remove_columns=test_dataset.column_names,
    num_proc=data_args.preprocessing_num_workers,

)

feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1,
    sampling_rate=16_000,
    padding_value=0.0,
    do_normalize=True,
    return_attention_mask=True,
)
processor = CustomWav2Vec2Processor(feature_extractor=feature_extractor)

def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."
    batch["input_values"] = processor(
        batch["speech"], sampling_rate=batch["sampling_rate"][0]
    ).input_values
    batch["labels"] = batch["parent"]
    return batch

train_dataset = train_dataset.map(
    prepare_dataset,
    remove_columns=train_dataset.column_names,
    batch_size=training_args.per_device_train_batch_size,
    batched=True,
    num_proc=data_args.preprocessing_num_workers,
)
eval_dataset = eval_dataset.map(
    prepare_dataset,
    remove_columns=eval_dataset.column_names,
    batch_size=training_args.per_device_train_batch_size,
    batched=True,
    num_proc=data_args.preprocessing_num_workers,
)
test_dataset = test_dataset.map(
    prepare_dataset,
    remove_columns=test_dataset.column_names,
    batch_size=training_args.per_device_train_batch_size,
    batched=True,
    num_proc=data_args.preprocessing_num_workers,
)

labels are: {'البسيط': 0, 'الخفيف': 1, 'الرجز': 2, 'الرمل': 3, 'السريع': 4, 'الطويل': 5, 'الكامل': 6, 'المتدارك': 7, 'المتقارب': 8, 'المجتث': 9, 'المديد': 10, 'المضارع': 11, 'المقتضب': 12, 'المنسرح': 13, 'الهزج': 14, 'الوافر': 15}
len: 16


  0%|          | 0/2934 [00:00<?, ?ex/s]

  0%|          | 0/367 [00:00<?, ?ex/s]

  0%|          | 0/367 [00:00<?, ?ex/s]

  0%|          | 0/1467 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


  0%|          | 0/184 [00:00<?, ?ba/s]

  0%|          | 0/184 [00:00<?, ?ba/s]

## check for previous checkpoints

In [ ]:
# Detecting last checkpoint.
last_checkpoint = None
if (
    os.path.isdir(training_args.output_dir)
    and training_args.do_train
    and not training_args.overwrite_output_dir
):
    last_checkpoint = get_last_checkpoint(training_args.output_dir)
    if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. "
            "Use --overwrite_output_dir to overcome."
        )
    elif last_checkpoint is not None:
        logger.info(
            f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
            "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
        )

## prepare the model

In [ ]:
pretrained_model_path = '/content/drive/MyDrive/KFUPM-Master/ICS606/Models/SinaiFineTuned/checkpoint-1600'
model = Wav2Vec2ClassificationModel.from_pretrained(
    pretrained_model_path,
    attention_dropout=0.01,
    hidden_dropout=0.01,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.01,
    num_attention_heads=4,
)

if model_args.freeze_feature_extractor:
    model.freeze_feature_extractor()

def compute_metrics(pred):
    labels = pred.label_ids.argmax(-1)
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    report = classification_report(labels, preds)
    matrix = confusion_matrix(labels, preds)
    # print(matrix)
    # print('clasification report',report)
    # print('accuracy',acc)
    return {"accuracy": acc}

# Data collator
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
# Initialize our Trainer
trainer = CTCTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset if training_args.do_train else None,
    eval_dataset=eval_dataset if training_args.do_eval else None,
    tokenizer=processor.feature_extractor,
)

loading configuration file /content/drive/MyDrive/KFUPM-Master/ICS606/Models/SinaiFineTuned/checkpoint-1600/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "bakrianoo/sinai-voice-ar-stt",
  "activation_dropout": 0.0,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.01,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",
  "feat_proj_dropout": 0.0,
  "feat_quantizer_drop

## training

In [ ]:
# Training
if training_args.do_train:
  if last_checkpoint is not None:
      checkpoint = last_checkpoint
  elif os.path.isdir(model_args.model_name_or_path):
      checkpoint = model_args.model_name_or_path
  else:
      checkpoint = None
  train_result = trainer.train(resume_from_checkpoint=checkpoint)
  trainer.save_model()

  # save the feature_extractor and the tokenizer
  if is_main_process(training_args.local_rank):
      processor.save_pretrained(training_args.output_dir)

  metrics = train_result.metrics
  max_train_samples = (
      data_args.max_train_samples
      if data_args.max_train_samples is not None
      else len(train_dataset)
  )
  metrics["train_samples"] = min(max_train_samples, len(train_dataset))

  trainer.log_metrics("train", metrics)
  trainer.save_metrics("train", metrics)
  trainer.save_state()

# Evaluation
results = {}
if training_args.do_eval:
  logger.info("*** Evaluate ***")
  metrics = trainer.evaluate()
  max_val_samples = (
      data_args.max_val_samples
      if data_args.max_val_samples is not None
      else len(eval_dataset)
  )
  metrics["eval_samples"] = min(max_val_samples, len(eval_dataset))

  trainer.log_metrics("eval", metrics)
  trainer.save_metrics("eval", metrics)

***** Running training *****
  Num examples = 2934
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 7335
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:986: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, roundin

Step,Training Loss,Validation Loss,Accuracy
100,2.581300,2.247297,0.297003
200,2.259300,2.241882,0.256131
300,2.110200,1.721183,0.490463
400,1.760300,1.735919,0.558583
500,1.578800,1.336853,0.607629
600,1.423800,1.469066,0.580381
700,1.411600,1.304375,0.626703
800,1.205200,1.595283,0.550409
900,1.249400,0.914040,0.713896
1000,1.317300,0.975664,0.705722


***** Running Evaluation *****
  Num examples = 367
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

KeyboardInterrupt: ignored

In [ ]:
# use it if you face problems of checkpoints not being saved
from google.colab import drive
drive.flush_and_unmount()